In [25]:
#import modules
from EDA import Preprocessing
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [26]:
#read datasets
import pandas as pd

df_mat = pd.read_csv('/home/pirate/Documents/machine_learning/score_prediction/student/student-mat.csv',sep=';')
df_por = pd.read_csv('/home/pirate/Documents/machine_learning/score_prediction/student/student-por.csv',sep=';')


In [27]:
#Handling duplicates found in datasets
df_merged=df_mat.append(df_por, ignore_index=True)
no_of_duplicates = df_merged.duplicated(["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"]).sum()
print(f'---> We have {no_of_duplicates} found in our two datasets\n')

print('---> Merging datasets and removing duplicates\n')
df_merged.drop_duplicates(subset=["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"],
                 keep = 'first', inplace = True)

print(f'---> After merging and removing duplicates we have {len(df_merged)} unique rows')


---> We have 382 found in our two datasets

---> Merging datasets and removing duplicates

---> After merging and removing duplicates we have 662 unique rows


In [28]:
# performing EDA
#check for missing values
Preprocessing(df_merged).check_missing_value()